In [1]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl
import shutil

import flopy
from modflowapi import ModflowApi

from bmi.wrapper import BMIWrapper

from liss_settings import libmf6, get_dflow_grid_name, get_dflow_dtuser, get_modflow_coupling_tag


# SWMM Package imports
import platform
import flopy.utils.binaryfile as bf
from flopy.utils.postprocessing import (
    get_gradients,
    get_transmissivities,
    get_water_table,
)

from modflowapi.extensions import ApiSimulation
import pyswmm
from pyswmm import Simulation, Nodes
from pyswmm import Output

c:\Users\lherdman\AppData\Local\anaconda3\envs\liss\lib\site-packages\bmi\__init__.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
control_path = pl.Path("../dflow-fm/coarse/tides/base/FlowFM.mdu") # change this if using a different D-Flow FM control file
grid_name = get_dflow_grid_name(control_path)
print(grid_name)

LIS_modflow_bathy2_net


In [3]:
dflowfm_dtuser = get_dflow_dtuser(control_path)
print(dflowfm_dtuser)

300.0


#### Set unit conversion factors

In [4]:
d2sec = 24. * 60. * 60.
hrs2sec = 60. * 60. 
m2ft = 3.28081
cfd2cms = 1.0 / ((3.28082**3) * 86400.)

#### Set the MODFLOW coupling frequency

Change the `mf_couple_freq_hours` value. Only tested for multiple of the D-Flow FM DtUser variable. Will not work for `mf_couple_freq_hours` values greater than 24.

In [5]:
mf_couple_freq_hours = 0.5  #Change this value to change the coupling frequency
mf_couple_freq = mf_couple_freq_hours * hrs2sec
dflow_per_mf = int(mf_couple_freq / dflowfm_dtuser)

In [6]:
print(f"MODFLOW coupling frequency {mf_couple_freq_hours} hours\nMODFLOW coupled to D-FLOW FM every {dflow_per_mf} output time step ({dflowfm_dtuser} sec.)") 

MODFLOW coupling frequency 0.5 hours
MODFLOW coupled to D-FLOW FM every 6 output time step (300.0 sec.)


In [7]:
mf_tag = get_modflow_coupling_tag(mf_couple_freq_hours)
print(f"MODFLOW coupling tag: {mf_tag}")

MODFLOW coupling tag: 30.00M


In [8]:
nstp = int(86400.0 / (dflow_per_mf * dflowfm_dtuser))
print(f"MODFLOW time steps per day: {nstp}")

MODFLOW time steps per day: 48


#### Set a few variables for controlling coupling

In [9]:
HDRY = -1e30
DEPTH_MIN = 0.1

#### Print the path of the modflow6 shared library

In [10]:
str(libmf6), libmf6.is_file()

('E:\\LIS\\Code\\nywsc_compound_flooding\\modflow\\mf6dll\\libmf6.dll', True)

#### Load the D-FLOW to MODFLOW weights

In [11]:
npzfile = np.load(f"../mapping/dflow2mfghb_{grid_name}.npz")
npzfile

NpzFile '../mapping/dflow2mfghb_LIS_modflow_bathy2_net.npz' with keys: dflow2mfghb, ghbmask, ghb2qext

In [12]:
dflow2mfghb = npzfile["dflow2mfghb"]
ghbmask = npzfile["ghbmask"]
ghb2qext = npzfile["ghb2qext"]

#### Define paths for the model simulation

In [13]:
mf_base_path = pl.Path("../modflow/greenport500ft/base/").resolve()
mf_run_path = pl.Path(f"../modflow/greenport500ft/run_{mf_tag}/").resolve()

#### Load the base MODFLOW model

In [14]:
sim = flopy.mf6.MFSimulation.load(sim_ws=mf_base_path)
gwf = sim.get_model()

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package npf...
    loading package obs...
    loading package sto...
    loading package oc...
    loading package ghb...
    loading package wel...
    loading package drn...
    loading package rch...
    loading package wel...
  loading solution package modflow...


In [15]:
sim.set_sim_path(mf_run_path)

#### Change TDIS to hourly time steps

In [16]:
tdis = sim.get_package("TDIS")
perioddata = tdis.perioddata.array

In [17]:
perioddata["nstp"] = nstp
tdis.perioddata = perioddata

#### Write the new model files

In [18]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package modflow...
  writing model modflow...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package obs-1...
    writing package sto...
    writing package oc...
    writing package ghb_obs...
    writing package ghb-1...
    writing package wel-1...
    writing package drn_obs...
    writing package drn-1...
    writing package rch-1...
    writing package swmm...


#### Define base GHB variables

In [19]:
ghb_data0 = gwf.ghb.stress_period_data.get_dataframe()[0]
ghb_data0

,cellid_layer,cellid_row,cellid_column,bhead,cond,iface,boundname
0,0,26,0,0.0,24999.999994,0,'sound'
1,0,27,0,0.0,25000.000000,0,'sound'
2,0,28,0,0.0,24999.999994,0,'sound'
3,0,29,0,0.0,24231.956659,0,'sound'
4,0,25,1,0.0,25000.000006,0,'sound'
...,...,...,...,...,...,...,...
730,0,14,56,0.0,22494.219708,0,'peconic'
731,0,15,56,0.0,25000.000006,0,'peconic'
732,0,16,56,0.0,24999.999994,0,'peconic'
733,0,17,56,0.0,25000.000000,0,'peconic'


#### Setup and initialize D-FLOW FM

You will need to set `dflow_dirpath` to the correct directory on your machine.

In [20]:
# dflow_dirpath = os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\bin")
# dflow_deps_dirpath = (
#     os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\share\bin"),
# )
#dflow_dirpath = pl.Path(r"E:\LIS\Code\nywsc_compound_flooding\dflow-fm\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\bin") 
dflow_dirpath=pl.Path(r"E:\LIS\Code\nywsc_compound_flooding\dflow-fm\dll_dir")
dflow_base = pl.Path(r"E:\LIS\Code\nywsc_compound_flooding\dflow-fm\coarse\tides\base")
dflow_working = pl.Path(r"E:\LIS\Code\nywsc_compound_flooding\dflow-fm\coarse\tides\run")
dflow_config = dflow_working / "FlowFM.mdu"

In [21]:
if dflow_working.is_dir():
    shutil.rmtree(dflow_working)
shutil.copytree(dflow_base, dflow_working)

WindowsPath('E:/LIS/Code/nywsc_compound_flooding/dflow-fm/coarse/tides/run')

In [22]:
os.environ["PATH"]

'c:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\Library\\mingw64\\bin;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\Library\\mingw-w64\\bin;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\Library\\usr\\bin;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\Library\\bin;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\Scripts;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\bin;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\condabin;C:\\Program Files\\Amazon Corretto\\jdk11.0.25_9\\bin;C:\\Program Files\\Amazon Corretto\\jdk1.8.0_432\\bin;C:\\Program Files (x86)\\Amazon Corretto\\jdk1.8.0_432\\bin;C:\\Program Files\\Amazon Corretto\\jdk21.0.4_7\\bin;C:\\Program Files\\Amazon Corretto\\jdk11.0.24_8\\bin;C:\\Python311\\Scripts;C:\\Python311;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C

In [23]:
# Add dflowfm dll folder to PATH so that it can be found by the BMIWrapper
os.environ["PATH"] = (
    str(dflow_dirpath) + os.pathsep + os.environ["PATH"]
)

In [24]:
str(dflow_dirpath)

'E:\\LIS\\Code\\nywsc_compound_flooding\\dflow-fm\\dll_dir'

In [25]:
os.environ["PATH"]

'E:\\LIS\\Code\\nywsc_compound_flooding\\dflow-fm\\dll_dir;c:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\Library\\mingw64\\bin;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\Library\\mingw-w64\\bin;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\Library\\usr\\bin;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\Library\\bin;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\Scripts;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\envs\\liss\\bin;C:\\Users\\lherdman\\AppData\\Local\\anaconda3\\condabin;C:\\Program Files\\Amazon Corretto\\jdk11.0.25_9\\bin;C:\\Program Files\\Amazon Corretto\\jdk1.8.0_432\\bin;C:\\Program Files (x86)\\Amazon Corretto\\jdk1.8.0_432\\bin;C:\\Program Files\\Amazon Corretto\\jdk21.0.4_7\\bin;C:\\Program Files\\Amazon Corretto\\jdk11.0.24_8\\bin;C:\\Python311\\Scripts;C:\\Python311;C:\\W

In [26]:
(pl.Path(dflow_dirpath) / "dflowfm.dll").is_file()

True

#### Initialize D-Flow FM

In [27]:
dflowfm = BMIWrapper(
    engine="dflowfm",
    configfile=os.path.abspath(dflow_config),
)

In [28]:
dflowfm.initialize()

#### Get data from D-FLOW FM

In [29]:
ndxi = int(dflowfm.get_var("ndxi"))
ndx = int(dflowfm.get_var("ndx"))
x = dflowfm.get_var("xz")
y = dflowfm.get_var("yz")
z = dflowfm.get_var("bl")
xy = [(xx, yy) for (xx, yy) in zip(x, y)]
ndx, ndxi, x.shape, y.shape

(8412, 8323, (8412,), (8412,))

In [30]:
dflowfm.get_var("s1")

array([0., 0., 0., ..., 0., 0., 0.])

In [31]:
v = dflowfm.get_var("hs")
v.shape, v

((8412,),
 array([26.42575318, 24.9297905 , 23.53244183, ..., 18.71800344,
        20.31243107, 21.52212007]))

In [32]:
qext = np.zeros(ndx)
qext.shape, qext

((8412,), array([0., 0., 0., ..., 0., 0., 0.]))

In [33]:
# dflowfm.set_var("qext", qext)

In [34]:
qext_cum = np.zeros(ndx)
qext_cum.shape

(8412,)

In [35]:
vextcum = dflowfm.get_var("vextcum")
vextcum.shape, vextcum

((8412,), array([0., 0., 0., ..., 0., 0., 0.]))

#### Initialize MODFLOW using MODFLOW API

In [36]:
mf6 = ModflowApi(str(libmf6), working_directory=mf_run_path)

In [37]:
mf6.initialize()

#### Define MODFLOW variable tags and set pointer to MODFLOW variables

In [38]:
ghb_bhead_tag = mf6.get_var_address("BHEAD", "MODFLOW", "GHB-1")
ghb_cond_tag = mf6.get_var_address("COND", "MODFLOW", "GHB-1")
ghb_flow_tag = mf6.get_var_address("SIMVALS", "MODFLOW", "GHB-1")

In [39]:
ghb_bhead_ptr = mf6.get_value_ptr(ghb_bhead_tag)
ghb_cond_ptr = mf6.get_value_ptr(ghb_cond_tag)
ghb_flow = np.zeros(ghb_bhead_ptr.shape)

#### Create dictionaries for saving modified GHB data

In [40]:
elev_dict = {}
cond_dict = {}
qext_dict = {}

#### Function to update MODFLOW GHB data

In [41]:
def update_mf(key, s, d):
    mask = d == 0.0
    s[mask] = 0.0
    mult = np.full(d.shape, 1.0)
    mult[mask] = 0.0
    
    ghb_head = ghb_data0["bhead"].to_numpy()
    ghb_head[ghbmask] = dflow2mfghb.dot(s)[ghbmask] * m2ft
    ghb_cond = ghb_data0["cond"].to_numpy()
    ghb_cond[ghbmask] = ghb_cond[ghbmask] * dflow2mfghb.dot(mult)[ghbmask]
    ghb_bhead_ptr[:] = ghb_head[:]
    ghb_cond_ptr[:] = ghb_cond[:]

    # update results dictionary
    elev_dict[key] = ghb_head.copy()
    cond_dict[key] = ghb_cond.copy()

#### Function to update D-Flow FM Qext data

In [42]:
def update_dflow(key, d):
    print("DFLOW DEBUGA") 
    ghb_flow = -mf6.get_value(ghb_flow_tag) * cfd2cms
    dflow_qext = ghb2qext.dot(ghb_flow)
    dflow_qext[d == 0.0] = 0.0
    qext_cum[:ndxi] += dflow_qext[:ndxi]
    print("DFLOW DEBUG1")  
    qext[:ndxi] = dflow_qext[:ndxi]
    dflowfm.set_var("qext", qext)

    # update results dictionaries
    qext_dict[key] = qext[:ndxi].copy()


SWMM INPUTS AND INITIALIZATION

In [43]:
# SWMM mappings
junctions = (
     "9",
     "10",
     "47",
     "54",
     "44",
     "34",
     "81",
     "155",
     "67",
     "69",
     "30",
     "118",
)

# these are the junction elevations in the SWMM inp file
swmm_inverts = {
    "9": 5,
    "10": 2,
    "47": 2.2,
    "54": 0.7,
    "44": 2.5,
    "34": 6.0,
    "81": 0.7,
    "155": 3,
    "67": -1.0,
    "69": 1.0,
    "30":  5.5,
    "118": 4.6,
 }
swmm_nodes = {}

# MODFLOW -----------------
# map of the junction names and the MODFLOW (row, column)
# containing the junction
mf6_cells = {
    "9": (25, 32),
    "10": (24, 32),
    "47": (28, 33),
    "54": (29, 33),
    "44": (27, 33),
    "34": (28, 31),
    "81": (28, 30),
    "155": (28, 29),
    "67": (31, 32),
    "69": (31, 31),
    "30": (27, 31),
    "118": (35, 26),
}

In [44]:


# use recent modflowapi functionality for accessing data from the api
# in flopy-like data structures
apisim = ApiSimulation.load(mf6)
apiml = apisim.get_model()

# get a handle to the well package for the SWMM flows
sewer_flow = apiml.get_package("SWMM")

current_time = dflowfm.get_current_time()
end_time = dflowfm.get_end_time()

In [45]:
# SWMM ------------------
ts = 60  # s, time step
t15d = ts * 24 * 15 * 60  # s, step advance of swmm = stress period
tmax = 24 * 60 * (end_time + 1)  # min
swmm_time = np.arange(1, tmax, 1)  # min
iswmm = 1  # index in the swmm simulation arrays
#simsw = pyswmm.Simulation("Greenport_Model_New_v1.inp")
#simsw=pyswmm.Simulation("greenport_detailedsewer_v4_nosub175.inp")
simsw=pyswmm.Simulation(r'E:\LIS\Code\nywsc_compound_flooding\swmm\greenport\greenport_detailedsewer_v4_nosub175.inp')

# build the swmm_nodes dictionary where the values are the swmm node objects
# for example, "J1 = pyswmm.Nodes(simsw)["J1"]"" , where J1 is the key
# and the value is pyswmm.Nodes(simsw)["J1"]
for j in junctions:
    swmm_nodes[j] = pyswmm.Nodes(simsw)[j]

Out1 = pyswmm.Nodes(simsw)["171"] # This is the junction at the WWTP
out_array = np.zeros(3650000)  # vector to record flow rate


In [48]:
simsw.start()

In [49]:
time_vec = np.zeros(365)

#### Run each time step

In [ ]:
print(
    f"DFLOWFM current_time: {dflowfm.get_current_time():15,.1f} sec. ({dflowfm.get_current_time()/86400.:15,.1f} days)\n"
     + f"DFLOWFM end_time:     {dflowfm.get_end_time():15,.1f} sec. ({dflowfm.get_end_time()/86400.:15,.1f} days)"
)

In [ ]:
idx = 0
jdx = 0
t0 = time.perf_counter()
current_time = dflowfm.get_current_time()
end_time = dflowfm.get_end_time()
j = 0
while current_time <= end_time:
    idx += 1
    ontime = dflowfm.get_current_time()
    onday = ontime / 86400.
    frac_comp = current_time / end_time
    print(f"Current time: {current_time:15,.1f} ({onday:10.3f} days) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
    dflowfm.update()

    current_time = dflowfm.get_current_time()
    if idx == int(dflow_per_mf):
        print(f"Current time: {current_time:15,.1f} ({onday:10.3f} days) - {frac_comp:6.2%} complete - ({idx:03d}) ***", end="\r")
        s = dflowfm.get_var("s1")[:ndxi]
        d = dflowfm.get_var("hs")[:ndxi]

        mf6.prepare_time_step(mf6.get_time_step())
        update_mf(str(jdx), s, d)
        mf6.do_time_step()
        mf6.finalize_time_step()
        update_dflow(str(jdx), d)

        idx = 0
        jdx += 1

        #SWMM
        dt = mf6.get_time_step()
        print(dt)
        Acurrent_time = mf6.get_current_time()
    
        heads = apiml.X
        #time_vec[j] = np.array(Acurrent_time)

        # Infiltrations and exfiltrations
        mf6_spd = []
        for key, value in swmm_nodes.items():
            row, col = mf6_cells[key]
            head = heads[0, row, col]
            # might be nice to use the simulated swmm water-level
            # in the junctions instead of the invert elevation
            pot = head - swmm_inverts[key] / 0.304  # ft
            if pot > 0.0:
                Q = pot * 0.001
            else:
                Q = pot * 0.0002
            value.generated_inflow(Q)  # CMS
            mf6_spd.append(((0, row, col), -Q))
            #print(Q)
            # Extracting the water table for 2D mapping
            #if round(time_vec[j]) == 100:
            #    headplt1=heads
    
        # update SWMM well file with new flux data - will be
        # used next time step
        dtype = [("nodelist", "O"), ("q", float)]

        sewer_flow.stress_period_data.values = np.array(
            mf6_spd, dtype=dtype
            )
        # SWMM -----------------------------------------------
        simsw.step_advance(dt * 3600 * 24)
        #out_array[j] = np.array(Out1.total_inflow)
        j += 1
        #
        print(f"finished...{simsw.current_time}")
t1 = time.perf_counter()    
print(f"\nrun time: {(t1 - t0) / 60.} min")

In [52]:
# Finalize
mf6.finalize()

In [53]:
simsw.close()

In [54]:
dflowfm.finalize()

#### Save ghb elevation and conductance data to compressed files

In [61]:
np.savez_compressed(f"{mf_run_path}/ghb_elev.npz", **elev_dict)
np.savez_compressed(f"{mf_run_path}/ghb_cond.npz", **cond_dict)
np.savez_compressed(f"{mf_run_path}/qext.npz", **qext_dict)